In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from data_tools import get_data_loaders
from models import DNNWithLASSO, MM_unrolled, MM_deq
from fixed_point_solvers import forward_iteration, anderson
from utils import train_model, lasso_regression_error_fixed_lambda

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
dataset_params = {"num_entries": 10000,
                  "n": 200,
                  "p": 100,
                  "decay_rate": 5,
                  "noise_std": 1.0}

hparams = {"batch_size": 32,
           "learning_rate": 1e-3,
           "num_epochs": 5,
           "hidden_dims": [128, 64, 32],
           "input_size": dataset_params['n']*dataset_params['p'] + dataset_params['n']}

train_loader, test_loader, train_indices, test_indices = get_data_loaders(dataset_params)

In [3]:
# Differentiable Convex Optimization Layers

model_dcol = DNNWithLASSO(hparams, dataset_params).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model_dcol.parameters(), lr=hparams["learning_rate"])
train_losses_dcol, test_losses_dcol = train_model(model_dcol, train_loader, test_loader, criterion, optimizer, num_epochs=hparams["num_epochs"], device=device)

/home/avp/anaconda3/envs/pytorchgpu/lib/python3.12/site-packages/cvxpy/reductions/solvers/solving_chain.py:254: UserWarning: Your problem has too many parameters for efficient DPP compilation. We suggest setting 'ignore_dpp = True'.
  warnings.warn(
Epoch 1/5: 100%|██████████| 250/250 [02:22<00:00,  1.76it/s]


Epoch [1/5], Running Loss: 0.005255
Train Loss: 0.008499
Test Loss: 0.004400


Epoch 2/5: 100%|██████████| 250/250 [02:22<00:00,  1.76it/s]


Epoch [2/5], Running Loss: 0.005699
Train Loss: 0.004925
Test Loss: 0.004656


Epoch 3/5: 100%|██████████| 250/250 [02:22<00:00,  1.75it/s]


Epoch [3/5], Running Loss: 0.004492
Train Loss: 0.003232
Test Loss: 0.004973


Epoch 4/5: 100%|██████████| 250/250 [02:23<00:00,  1.75it/s]


Epoch [4/5], Running Loss: 0.003200
Train Loss: 0.002938
Test Loss: 0.005067


Epoch 5/5: 100%|██████████| 250/250 [02:23<00:00,  1.75it/s]


Epoch [5/5], Running Loss: 0.002899
Train Loss: 0.002814
Test Loss: 0.005662
Average memory stored from forward pass: 2.62 MB


In [4]:
# Unrolled MM

model_unrolled = DNNWithLASSO(hparams, dataset_params, MM_func=MM_unrolled).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model_unrolled.parameters(), lr=hparams["learning_rate"])
train_losses_unrolled, test_losses_unrolled = train_model(model_unrolled, train_loader, test_loader, criterion, optimizer, num_epochs=hparams["num_epochs"], device=device)

Epoch 1/5: 100%|██████████| 250/250 [00:44<00:00,  5.60it/s]


Epoch [1/5], Running Loss: 0.005423
Train Loss: 0.003362
Test Loss: 0.004800


Epoch 2/5: 100%|██████████| 250/250 [00:44<00:00,  5.65it/s]


Epoch [2/5], Running Loss: 0.003350
Train Loss: 0.002902
Test Loss: 0.004828


Epoch 3/5: 100%|██████████| 250/250 [00:45<00:00,  5.55it/s]


Epoch [3/5], Running Loss: 0.002897
Train Loss: 0.002792
Test Loss: 0.004866


Epoch 4/5: 100%|██████████| 250/250 [00:45<00:00,  5.48it/s]


Epoch [4/5], Running Loss: 0.002787
Train Loss: 0.002789
Test Loss: 0.004882


Epoch 5/5: 100%|██████████| 250/250 [00:45<00:00,  5.52it/s]


Epoch [5/5], Running Loss: 0.002782
Train Loss: 0.002807
Test Loss: 0.004870
Average memory stored from forward pass: 18.26 MB


In [6]:
# Implicit MM. Forward iteration solver

model_implicit_forward = DNNWithLASSO(hparams, dataset_params, MM_func=MM_deq, fixed_point_solver=forward_iteration).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model_implicit_forward.parameters(), lr=hparams["learning_rate"])
train_losses_implicit_forward, test_losses_implicit_forward = train_model(model_implicit_forward, train_loader, test_loader, criterion, optimizer, num_epochs=hparams["num_epochs"], device=device)

Epoch 1/5: 100%|██████████| 250/250 [00:34<00:00,  7.17it/s]


Epoch [1/5], Running Loss: 0.006023
Train Loss: 0.003580
Test Loss: 0.004926


Epoch 2/5: 100%|██████████| 250/250 [00:34<00:00,  7.32it/s]


Epoch [2/5], Running Loss: 0.003565
Train Loss: 0.002955
Test Loss: 0.004803


Epoch 3/5: 100%|██████████| 250/250 [00:34<00:00,  7.25it/s]


Epoch [3/5], Running Loss: 0.002946
Train Loss: 0.002801
Test Loss: 0.004910


Epoch 4/5: 100%|██████████| 250/250 [00:34<00:00,  7.20it/s]


Epoch [4/5], Running Loss: 0.002793
Train Loss: 0.002782
Test Loss: 0.004885


Epoch 5/5: 100%|██████████| 250/250 [00:34<00:00,  7.27it/s]


Epoch [5/5], Running Loss: 0.002780
Train Loss: 0.002795
Test Loss: 0.004922
Average memory stored from forward pass: 2.77 MB


In [7]:
# Implicit MM. Anderson solver
model_implicit_anderson = DNNWithLASSO(hparams, dataset_params, MM_func=MM_deq, fixed_point_solver=anderson).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model_implicit_anderson.parameters(), lr=hparams["learning_rate"])
train_losses_implicit_anderson, test_losses_implicit_anderson = train_model(model_implicit_anderson, train_loader, test_loader, criterion, optimizer, num_epochs=hparams["num_epochs"], device=device)

Epoch 1/5: 100%|██████████| 250/250 [01:22<00:00,  3.02it/s]


Epoch [1/5], Running Loss: 0.005326
Train Loss: 0.003349
Test Loss: 0.004700


Epoch 2/5: 100%|██████████| 250/250 [01:23<00:00,  3.00it/s]


Epoch [2/5], Running Loss: 0.003305
Train Loss: 0.002912
Test Loss: 0.004728


Epoch 3/5: 100%|██████████| 250/250 [01:24<00:00,  2.96it/s]


Epoch [3/5], Running Loss: 0.002896
Train Loss: 0.002791
Test Loss: 0.004832


Epoch 4/5: 100%|██████████| 250/250 [01:26<00:00,  2.90it/s]


Epoch [4/5], Running Loss: 0.002789
Train Loss: 0.002816
Test Loss: 0.004821


Epoch 5/5: 100%|██████████| 250/250 [01:24<00:00,  2.95it/s]


Epoch [5/5], Running Loss: 0.002794
Train Loss: 0.002824
Test Loss: 0.004843
Average memory stored from forward pass: 2.77 MB


In [8]:
# LASSO regression with a fixed lambda value

# First, the value of 0.05 was determined using the train_loader
lasso_regression_error_fixed_lambda(0.05, test_loader)

(Lasso regression - Test Set) Average MSE with L1 penalty: 0.003893
